In [1]:
#### json -> QB ####

In [32]:
# modules and libraries:
import itertools, json, pandas as pd, numpy as np

In [33]:
# open and read .json:
def open_json():
    with open("PDM_json_to_test.json",
                  mode = "r",
                  encoding = "utf8") as f:
        data = json.loads(f.read())
    return data

open_json()

{'type': 'root',
 'children': [{'identifier': 'enumerator_id',
   'title': 'Enumerator ID',
   'type': 'text',
   'required_rule': 'always'},
  {'identifier': 'consent_text',
   'title': 'My name is [Enumerator name] and I work for the [International Committee of the Red Cross / Name of National Society]. [ICRC / NS on behalf of the ICRC] is collecting [type of data] from [type of respondents] in [Area/Country]. We would like to ask you some questions about your [type of data]. The purpose of collecting this information is to support [ICRC and/or NS]’s operational planning and decisions regarding its [type of intervention] to assist the most vulnerable population in the area. The survey usually takes [length in minutes] to complete. Any information that you provide will only be used by [ICRC and/or NS] and is not subject to further disclosure. Your participation is voluntary. We need your genuine responses and hope that you agree to participate.',
   'type': 'text',
   'readonly_rule':

In [34]:
# normalisation of .json:
def normalize():
    df = pd.json_normalize(open_json()["children"])
    return df

normalize().head(5)

,identifier,title,type,required_rule,readonly_rule,options,validate_expr,children,minOccurs,maxOccurs,visible_rule,visible_expr,multi_line
0,enumerator_id,Enumerator ID,text,always,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,consent_text,My name is [Enumerator name] and I work for th...,text,NaN,always,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,consent,Do you consent to being interviewed?,boolean,always,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,date,Date,date,always,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,name_of_the_form,Form name,select,always,NaN,"[{'text': 'Form name', 'identifier': 'Form nam...",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
# remove sufficient cols:
cols_to_del = ["minOccurs",
               "maxOccurs",
               "children",
               "autoIdentifier",
               "customOptionIdentifiers"]

df = normalize()
for a in cols_to_del:
    try:
        df = df.drop(columns=[a])
    except Exception:
        pass
    
df.head(5)

,identifier,title,type,required_rule,readonly_rule,options,validate_expr,visible_rule,visible_expr,multi_line
0,enumerator_id,Enumerator ID,text,always,NaN,NaN,NaN,NaN,NaN,NaN
1,consent_text,My name is [Enumerator name] and I work for th...,text,NaN,always,NaN,NaN,NaN,NaN,NaN
2,consent,Do you consent to being interviewed?,boolean,always,NaN,NaN,NaN,NaN,NaN,NaN
3,date,Date,date,always,NaN,NaN,NaN,NaN,NaN,NaN
4,name_of_the_form,Form name,select,always,NaN,"[{'text': 'Form name', 'identifier': 'Form nam...",NaN,NaN,NaN,NaN


In [36]:
# replace bools with Yes:
cols_to_replace = [
    "multi_line",
    "timestamp",
    "geostamp",
    "multiple"
]

for b in cols_to_replace:
    try:
        df[b].replace(True, "Yes",
                      regex = True,
                      inplace = True)
    except Exception:
        pass
    
df.head(5)

,identifier,title,type,required_rule,readonly_rule,options,validate_expr,visible_rule,visible_expr,multi_line
0,enumerator_id,Enumerator ID,text,always,NaN,NaN,NaN,NaN,NaN,NaN
1,consent_text,My name is [Enumerator name] and I work for th...,text,NaN,always,NaN,NaN,NaN,NaN,NaN
2,consent,Do you consent to being interviewed?,boolean,always,NaN,NaN,NaN,NaN,NaN,NaN
3,date,Date,date,always,NaN,NaN,NaN,NaN,NaN,NaN
4,name_of_the_form,Form name,select,always,NaN,"[{'text': 'Form name', 'identifier': 'Form nam...",NaN,NaN,NaN,NaN


In [37]:
# extract list of title subforms:
def get_sforms():
    groups = (df[df["type"] == "group"])["title"].to_list()
    return groups

get_sforms()

['Identity of the respondant',
 'Demographic Information',
 'Assistance verification',
 'Access to distribution point',
 'Access to market',
 'Programme site safety',
 'Communication',
 'Perception of Appropriateness of ICRC Assistance',
 'Perception of registration process and targeting',
 'Feedback mechanisms']

In [38]:
# split dataframe - each row as separate df:
def split_dataframe(df, num_splits):
    num_rows = len(df)
    rows_per_split = int(np.ceil(num_rows / num_splits))
    splits = np.array_split(df, num_splits)
    split_dataframes = []
    for split in splits:
        split_dataframes.append(split)
    return split_dataframes

split_dataframes = split_dataframe(df, len(df))
split_dataframes[1]

,identifier,title,type,required_rule,readonly_rule,options,validate_expr,visible_rule,visible_expr,multi_line
1,consent_text,My name is [Enumerator name] and I work for th...,text,NaN,always,NaN,NaN,NaN,NaN,NaN


In [39]:
# assign original df as main:
main_df = split_dataframes

# get list of single questions and subforms:
stru = []
acum = 0
for x in split_dataframes:
    if split_dataframes[acum]["type"].isin(["group"]).any():
        stru.append("subform")
        acum = acum + 1
    else:
        stru.append("single_question")
        acum = acum + 1
        
stru

['single_question',
 'single_question',
 'single_question',
 'single_question',
 'single_question',
 'single_question',
 'subform',
 'subform',
 'subform',
 'subform',
 'subform',
 'subform',
 'subform',
 'subform',
 'subform',
 'subform',
 'single_question']

In [40]:
# fetch nested data:
def dataframes():
    nested = open_json()["children"]
    result = []
    for x in range(len(nested)):
        try:
            df = pd.json_normalize(nested[x],
                 record_path = ["children"])
            result.append(df)
        except: pass
    return result

dataframes()[0].head(5)

,identifier,title,type,required_rule,options,multi_line
0,h_o_hh_name,What is the name of the head of household,text,always,NaN,NaN
1,UID,UID,text,always,NaN,NaN
2,weeks_since_distribution,Number of weeks since the distribution,integer,always,NaN,NaN
3,h_o_hh_phone,What is the phone number of the head of household,phone_number,always,NaN,NaN
4,adm0,Country,select,always,"[{'text': 'Burundi'}, {'text': 'CAR'}, {'text'...",NaN


In [42]:
# check subformed stru:
dataframes()

[                 identifier  \
 0               h_o_hh_name   
 1                       UID   
 2  weeks_since_distribution   
 3              h_o_hh_phone   
 4                      adm0   
 5                      adm1   
 6                      adm2   
 7            address_street   
 8            address_number   
 9             address_notes   
 
                                                title          type  \
 0          What is the name of the head of household          text   
 1                                                UID          text   
 2             Number of weeks since the distribution       integer   
 3  What is the phone number of the head of household  phone_number   
 4                                            Country        select   
 5                                             Region        select   
 6                                         Sub Region        select   
 7                              Address - Street name          text   
 8     

In [43]:
# assign subform title to each subformed question:
sub_df = dataframes()
acum = 0
for x, y in zip(sub_df, get_sforms()):
    sub_df[acum]["Parent Sub Form"] = y
    acum = acum + 1
    
sub_df[1].head(5)

,identifier,title,options,type,required_rule,customOptionIdentifiers,visible_rule,visible_expr,validate_expr,hint,Parent Sub Form
0,gender,Gender of respondent,"[{'text': 'Male', 'identifier': 'Musko'}, {'te...",select,always,True,NaN,NaN,NaN,NaN,Demographic Information
1,h_o_hh,Are you the head of household?,NaN,boolean,always,NaN,NaN,NaN,NaN,NaN,Demographic Information
2,h_o_hh_gender,"If no, what is the gender of the head of house...","[{'text': 'Male', 'identifier': 'Musko'}, {'te...",select,always,NaN,when,h_o_hh = false,NaN,NaN,Demographic Information
3,h_o_hh_age,How old is the head of household,NaN,integer,always,NaN,NaN,NaN,h_o_hh_age > 0,NaN,Demographic Information
4,h_o_hh_marital_status,What is the marital status of the head of hous...,"[{'text': 'Married', 'identifier': 'Married'},...",select,always,NaN,NaN,NaN,NaN,NaN,Demographic Information


In [44]:
# compare, align and merge main splitted df and
# subformed dfs into one:
def merge_dfs():
    final = []
    acum = 0
    for a, b, c in itertools.zip_longest(stru, main_df, sub_df):
        if a == "single_question":
            final.append(b)
        elif a == "subform":
            final.extend([b,sub_df[acum]])
            acum = acum + 1
    result = pd.concat(final)
    return result

final_df = merge_dfs()
final_df.head(5)

,identifier,title,type,required_rule,readonly_rule,options,validate_expr,visible_rule,visible_expr,multi_line,Parent Sub Form,customOptionIdentifiers,hint,multiple
0,enumerator_id,Enumerator ID,text,always,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,consent_text,My name is [Enumerator name] and I work for th...,text,NaN,always,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,consent,Do you consent to being interviewed?,boolean,always,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,date,Date,date,always,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,name_of_the_form,Form name,select,always,NaN,"[{'text': 'Form name', 'identifier': 'Form nam...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
# transform options:text part:
final_df["Options"] = (final_df["options"]
                    .apply(lambda x: ', '
                    .join([item["text"] for item in x]) if isinstance(x, list) else np.nan))

final_df.head(5)

,identifier,title,type,required_rule,readonly_rule,options,validate_expr,visible_rule,visible_expr,multi_line,Parent Sub Form,customOptionIdentifiers,hint,multiple,Options
0,enumerator_id,Enumerator ID,text,always,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,consent_text,My name is [Enumerator name] and I work for th...,text,NaN,always,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,consent,Do you consent to being interviewed?,boolean,always,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,date,Date,date,always,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,name_of_the_form,Form name,select,always,NaN,"[{'text': 'Form name', 'identifier': 'Form nam...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Form name


In [46]:
# transform options:identifiers part:
def process_row(row):
    if isinstance(row, list):
        identifiers = [item.get("identifier") for item in row]
        identifier_result = ', '.join(filter(None, identifiers))
        return identifier_result if identifier_result else np.nan
    else:
        return np.nan
    
final_df["Option Identifiers"] = final_df["options"].apply(process_row)
final_df.head(5)

,identifier,title,type,required_rule,readonly_rule,options,validate_expr,visible_rule,visible_expr,multi_line,Parent Sub Form,customOptionIdentifiers,hint,multiple,Options,Option Identifiers
0,enumerator_id,Enumerator ID,text,always,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,consent_text,My name is [Enumerator name] and I work for th...,text,NaN,always,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,consent,Do you consent to being interviewed?,boolean,always,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,date,Date,date,always,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,name_of_the_form,Form name,select,always,NaN,"[{'text': 'Form name', 'identifier': 'Form nam...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Form name,Form name


In [47]:
# delete sufficient cols from merged df:
cols_to_del_two = ["minOccurs",
                   "maxOccurs",
                   "children",
                   "autoIdentifier",
                   "customOptionIdentifiers",
                   "options",
                   "visible_rule"]

for a in cols_to_del_two:
    try:
        final_df = final_df.drop(columns=[a])
    except Exception:
        pass
    
final_df.head(5)

,identifier,title,type,required_rule,readonly_rule,validate_expr,visible_expr,multi_line,Parent Sub Form,hint,multiple,Options,Option Identifiers
0,enumerator_id,Enumerator ID,text,always,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,consent_text,My name is [Enumerator name] and I work for th...,text,NaN,always,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,consent,Do you consent to being interviewed?,boolean,always,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,date,Date,date,always,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,name_of_the_form,Form name,select,always,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Form name,Form name


In [48]:
# rename and relocate columns:
js = list(final_df.columns)
json_columns = js

final_columns = []
for x in json_columns:
    if x == "identifier":
        final_columns.append("Identifier")
    elif x == "title":
        final_columns.append("Question title")
    elif x == "required_rule":
        final_columns.append("Required")
    elif x == "timestamp":
        final_columns.append("Timestamp")
    elif x == "initialAnswer":
        final_columns.append("Initial value")
    elif x == "type":
        final_columns.append("Type")
    elif x == "validate_expr":
        final_columns.append("Validity expression")
    elif x == "calculate_expr":
        final_columns.append("Calculated expression")
    elif x == "type_derived":
        final_columns.append("Derived type")
    elif x == "geostamp":
        final_columns.append("Geostamp")
    elif x == "readonly_rule":
        final_columns.append("Read Only")
    elif x == "multi_line":
        final_columns.append("Long Text")
    elif x == "Parent Sub Form":
        final_columns.append("Parent Sub Form")
    elif x == "multiple":
        final_columns.append("Multi Choice")
    elif x == "hint":
        final_columns.append("Hint")
    elif x == "visible_expr":
        final_columns.append("Visibility expression")
    elif x == "Options":
        final_columns.append("Options")
    elif x == "Option Identifiers":
        final_columns.append("Option Identifiers")
        
final_df.columns = final_columns
final_df.head(5)

,Identifier,Question title,Type,Required,Read Only,Validity expression,Visibility expression,Long Text,Parent Sub Form,Hint,Multi Choice,Options,Option Identifiers
0,enumerator_id,Enumerator ID,text,always,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,consent_text,My name is [Enumerator name] and I work for th...,text,NaN,always,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,consent,Do you consent to being interviewed?,boolean,always,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,date,Date,date,always,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,name_of_the_form,Form name,select,always,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Form name,Form name


In [49]:
# relocate and and add missing columns:
origin = ['Question title', 'Identifier', 'Type', 'Options', 'Option Identifiers',
        'Parent Sub Form', 'Required', 'Read Only', 'Long Text', 'Multi Choice',
        'Timestamp', 'Geostamp', 'Hint', 'Visibility expression', 'Validity expression',
        'Calculated expression','Derived type', 'Initial value']

to_change = list(final_df.columns)
cols_to_add = list(set(origin) - set(to_change))

if len(cols_to_add) == 0:
    final_df = final_df[origin]
else:
    for x in cols_to_add:
        final_df[x] = np.nan
    final_df = final_df[origin]
    
final_df.head(5)

,Question title,Identifier,Type,Options,Option Identifiers,Parent Sub Form,Required,Read Only,Long Text,Multi Choice,Timestamp,Geostamp,Hint,Visibility expression,Validity expression,Calculated expression,Derived type,Initial value
0,Enumerator ID,enumerator_id,text,NaN,NaN,NaN,always,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,My name is [Enumerator name] and I work for th...,consent_text,text,NaN,NaN,NaN,NaN,always,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Do you consent to being interviewed?,consent,boolean,NaN,NaN,NaN,always,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Date,date,date,NaN,NaN,NaN,always,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Form name,name_of_the_form,select,Form name,Form name,NaN,always,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
# adjust bools and always options:
always = ["Required", "Read Only"]
bools = ["Long Text", "Multi Choice", "Timestamp", "Geostamp"]

for x in always:
    try:
        final_df[x].replace("always", "Yes",
                      regex = True,
                      inplace = True)
    except Exception:
        pass
    
for y in bools:
    try:
        final_df[y].replace(True, "Yes",
                      regex = True,
                      inplace = True)
    except Exception:
        pass
    
final_df.tail(5)

,Question title,Identifier,Type,Options,Option Identifiers,Parent Sub Form,Required,Read Only,Long Text,Multi Choice,Timestamp,Geostamp,Hint,Visibility expression,Validity expression,Calculated expression,Derived type,Initial value
15,Feedback mechanisms,feedback_mechanisms,group,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,If you have any suggestions or complaints abou...,icrc_feedback,boolean,NaN,NaN,Feedback mechanisms,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"If yes, how do you do this? (Do not read the o...",icrc_feedback_mechanism,select,"Speak to a volunteer, Write a letter, Go to th...","Speak to a volunteer, Write a letter, Go to th...",Feedback mechanisms,Yes,NaN,NaN,Yes,NaN,NaN,NaN,icrc_feedback = true,NaN,NaN,NaN,NaN
2,"If Other, specify",icrc_feedback_mechanism_other,text,NaN,NaN,Feedback mechanisms,Yes,NaN,NaN,NaN,NaN,NaN,NaN,"CONTAINS(icrc_feedback_mechanism,""Other"")",NaN,NaN,NaN,NaN
16,Do you have any additional feedback for the ICRC?,additional_comment,text,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
# final cleaning:
question_type_map = {
    'group':'Sub Form', 'select':'Select', 'signature':'Signature', 'image':'Image', 'time':'Time',
    'text':'Free Text', 'datetime':'Date+Time', 'email':'Email', 'barcode':'Barcode', 'decimal':'Decimal',
    'sketch':'Sketch', 'integer':'Integer', 'boolean':'Yes/No', 'calculated':'Calculated',
    'password':'Password', 'date':'Date', 'location':'Location', 'phone_number':'Phone No.' 
}

def final_adj():
    df = final_df
    df.fillna("",
              inplace = True)
    df["Type"].replace(question_type_map,
                       inplace = True)
    df["Options"] = (df["Options"]
                    .str.replace(", ", ","))
    df["Option Identifiers"] = (df["Option Identifiers"]
                               .str.replace(", ", ","))
    return df

final_adj().tail(5)

,Question title,Identifier,Type,Options,Option Identifiers,Parent Sub Form,Required,Read Only,Long Text,Multi Choice,Timestamp,Geostamp,Hint,Visibility expression,Validity expression,Calculated expression,Derived type,Initial value
15,Feedback mechanisms,feedback_mechanisms,Sub Form,,,,,,,,,,,,,,,
0,If you have any suggestions or complaints abou...,icrc_feedback,Yes/No,,,Feedback mechanisms,Yes,,,,,,,,,,,
1,"If yes, how do you do this? (Do not read the o...",icrc_feedback_mechanism,Select,"Speak to a volunteer,Write a letter,Go to the ...","Speak to a volunteer,Write a letter,Go to the ...",Feedback mechanisms,Yes,,,Yes,,,,icrc_feedback = true,,,,
2,"If Other, specify",icrc_feedback_mechanism_other,Free Text,,,Feedback mechanisms,Yes,,,,,,,"CONTAINS(icrc_feedback_mechanism,""Other"")",,,,
16,Do you have any additional feedback for the ICRC?,additional_comment,Free Text,,,,,,Yes,,,,,,,,,


In [22]:
# save prepared file:
def save_file():
    dff = final_adj()
    dff["Visibility expression"] = (dff["Visibility expression"]
                                    .str.replace('"',
                                    '\\"'))
    dff.to_csv(str(open_json()["title"]) + ".csv", index = False)
    
# calling final function:
save_file()

In [23]:
# end